# Setup
## Imports

In [1]:
import argparse
import numpy as np
import os
import pandas as pd
import pdb
import pickle as pkl
import random
import torch

from src.Transformer import retrieve_tf_names, GeneMasker, GRNInferModel
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

## Argument setup


In [10]:
opt = argparse.Namespace(
    alpha = 0.01, # scale hyperparameter for the loss regularisation term
    attention_dim = 64, # dimensionality of the attention layer
    attention_heads = 4, # number of attention heads
    batchsize = 32,
    dataset = "",
    data_file = "./demo_data/BEELINE/500_ChIP-seq_mDC/inputs/data.csv", # path of input scRNA-seq file
    dropout = 0.0, # Dropout probability in the dropout layers of the model
    dorothea_grade = "A", # the minimum allowable Dorothea letter grade for transcription factors
    embed_dim = 64, # embedding dimension
    ffn_embed_dim = 64, # embedding dimension for both feed forward networks of the model
    lr = 1e-4, # learning rate
    n_epochs = 1500, # number of epochs of training
    number_report_score = 4, # the minimum allowable Number Report score for transcription factors
    PIDC_file = "./demo_data/BEELINE/500_ChIP-seq_mDC/inputs/PIDC_output_file.txt",
    save_name = 'pretrain_output',
    seed = 0,
    tf_data_file = "transcription_factors.csv", # path of transcription factor csv file
    top_k = 20, # The number of largest attention weights to retain when calculating loss
)

## Seed

In [6]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available(): # GPU operation have separate seed
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

set_seed(opt.seed)

## Device

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    
print("Using device: ", device)

Using device:  cpu


# Initial Data Setup

In [11]:
def init_data(opt):

    # load data
    data = pd.read_csv(opt.data_file, header = 0, index_col = 0).T
    data_values = data.values

    ###########################################################################
    # this section is not required once we have a proper embedding to work with

    # build masks for data (nonzero values are true)
    d_mask_np = (data_values != 0).astype(float)
    d_mask = torch.FloatTensor(d_mask_np)

    # get mean and std for each column (excluding zero entries)
    means = []
    stds = []
    # loop through each column
    for i in range(data_values.shape[1]):
        tmp = data_values[:, i]

        # is there any non-zero entries in this column
        if sum(tmp != 0) == 0:
            # every entry is zero
            means.append(0)
            stds.append(1)
            
        else:
            # append mean and std of nonzero entries
            means.append(tmp[tmp != 0].mean())
            stds.append(tmp[tmp != 0].std())

    # convert to np.array
    means = np.array(means)
    stds = np.array(stds)

    # set any Nan or inf entries of means to 1
    means[np.isnan(stds)] = 0
    means[np.isinf(stds)] = 0

    # set any Nan, inf, or zero entries of stds to 1
    stds[(np.isnan(stds))] = 1
    stds[np.isinf(stds)] = 1
    stds[stds == 0] = 1
    
    # shift each column by it's mean and scale it by it's std
    data_values = (data_values - means) / (stds)

    # Set any nan or inf entries to 0
    data_values[np.isnan(data_values)] = 0
    data_values[np.isinf(data_values)] = 0

    # Set any entries less then -20 to -20
    data_values = np.maximum(data_values, -20)
    # Set any entries greater then 20 to 20
    data_values = np.minimum(data_values, 20)

    ###########################################################################

    # store as dataframe and float tensor
    data = pd.DataFrame(data_values, index = data.index, columns = data.columns)
    feat_train = torch.FloatTensor(data.values)

    return feat_train, d_mask, data.columns


input_all, d_mask, gene_name = init_data(opt)
tf_data = pd.read_csv(opt.tf_data_file)
tf_names = retrieve_tf_names(
    gene_name, 
    tf_data, 
    opt.dorothea_grade, 
    opt.number_report_score
)
print('Finished data preprocessing')


Finished data preprocessing


# Model Setup

In [12]:
# catch if we have an embedding of dimension 1
# we want a shape of [# cells, # genes, embedding dim]
if input_all.dim() == 2:
    # the embedding for each gene is 1
    input_all = input_all.unsqueeze(-1)
    model_input_dim = 1
    
else:
    model_input_dim = input_all.shape[-1]

# set up model
model = GRNInferModel(
    input_dim = model_input_dim, 
    attn_dim = opt.attention_dim,
    num_heads = opt.attention_heads, 
    pre_ffn_embed_dim = opt.ffn_embed_dim,
    post_ffn_embed_dim = opt.ffn_embed_dim,
    alpha = opt.alpha,
    top_k = opt.top_k,
    dropout = opt.dropout, 
).to(device)


## Masker setup

In [13]:
#setup masking object              
g_masker = GeneMasker(
    genes = gene_name,
    t_factors = tf_names
)

## Optimiser setup

In [14]:
# setup optimiser
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr = opt.lr, 
    betas=(0.9, 0.999), 
    weight_decay = 0.01
)

# Data Loader

In [15]:
#setup data loader
dataset = TensorDataset(
    input_all, 
    d_mask, 
    torch.LongTensor(list(range(len(input_all))))
)

drop_last = True
if len(input_all) < opt.batchsize:
    drop_last = False

dataloader = DataLoader(
    dataset, 
    batch_size = opt.batchsize, 
    shuffle = True, 
    num_workers = 1, 
    drop_last = drop_last
)

# Train

In [16]:
model.train()

loss_save = []
t = tqdm(range(opt.n_epochs))
for epoch in t: #tqdm(range(opt.n_epochs)):
    loss_all = []
    model = model.to(device)

    for X, mask, idn in dataloader:

        optimizer.zero_grad()
        X_clone = X.clone()
        X = X.to(device)

        # mask non-transcription factor genes in input X
        g_masker.mask_non_tfs(X)
                                
        # forward pass
        output, attn = model(X, return_attn = True)

        # calculate loss          
        # mask the transcription factors for the original input to the model and the models output
        g_masker.mask_tfs(output)
        g_masker.mask_tfs(X_clone)

        loss = model.loss(
            output, 
            X_clone.to(device), 
            attn 
        )

        # backward step
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), 0.001)
        optimizer.step()

        # record loss
        loss_all.append(loss)

    # update tqdm ticker
    t.set_description(f'Finished training epoch: {epoch}; loss: {torch.stack(loss_all).mean()}')

    # 
    loss_save.append(torch.stack(loss_all).mean().cpu().item())


Finished training epoch: 2; loss: 1.0868526697158813:   0%|          | 3/1500 [00:47<6:31:21, 15.69s/it]


KeyboardInterrupt: 

# Test

In [17]:
print('Begin generate candidates GRN features')
test_device = device 

model = model.to(test_device)
input = input_all.clone().to(test_device).unsqueeze(0)

with torch.no_grad():
    output, attn = model(input, return_attn = True)

adj = []
for i in range(attn.shape[-1]): 
    adj.append(attn[:, :, i].cup().numpy())

adj = np.array(adj)
pkl.dump([adj, loss_save], open(f'{opt.save_name}', 'wb'))


Begin generate candidates GRN features


ValueError: too many values to unpack (expected 3)